# 2 million datapoints from bsk

In [ ]:
from src.get_city_messages import get_city_messages
from src.classify_sentiment import classify_with_sentiment
from src.CHS_computation import compute_topic_signed_scores, normalize_topic_scores_0_10, compute_CHS

%load_ext autoreload
%autoreload 2

/home/ianyang/micromamba/envs/HTB2026/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Example
test_posts = [get_city_messages(5)]
for post in test_posts:
    print(post)

[{'text': '🟢 "Im Gedenken an die Opfer der Agenda 2010 (Hartz IV Tote), im Angesicht\nerneut verschärfter Sanktionspraxis und sonstiger Leistungsversagung(..)".\nHeute, 27.11.24 um 11.30 Uhr. Ort: Prenzlauer Allee 10405. #polizeiberlin\n#b2711 #berlin #berlinevents #demoberlin'}, {'text': "Was always a big radio listener. Stuck under a pair of daft sized giant red headphones that my Dad bought when I was young. I thank Charlie Gillet & Mike Sparrow (RIP both, Radio London), John Peel (natch, BBC Radio 1) & Nicky Horne (Capital Radio when Capital wasn't junk).\nThanks all. #music #radio"}, {'text': "Agree with every word of this 🧵. I also still expect it to be the defining factor that sees him sign a new contract (even if there remains a desire to move in a future window). Owen is offering advice on Madrid. I'd advise Trent to look more to what has followed for Michael. I think he will. #LFC"}, {'text': 'Med fem veckors framförhållning meddelar M:s och KD:s systerparti CDU en brutal sla

In [ ]:
# this is the social media posts, a list of dictionaries with "text" fields
# test_posts may be either:
#  - a list of post dicts: [{"text":...}, ...]
#  - or a list containing that list: [[{"text":...}, ...]]
# prefer a top-level variable `post` if present (it's a list of dicts in the notebook)
if "post" in globals() and isinstance(post, list) and post and isinstance(post[0], dict) and "text" in post[0]:
	posts = post
else:
	if isinstance(test_posts, list) and len(test_posts) == 1 and isinstance(test_posts[0], list):
		posts = test_posts[0]
	else:
		posts = test_posts

texts = [p["text"] for p in posts]

# compute topic signed scores (this is the raw scores in [-1, 1])
raw, details = compute_topic_signed_scores(texts, classify_with_sentiment, topk_mean=3, threshold=0.25)
# compute the 11 normalized topic scores between [0, 10]
topic_0_10 = normalize_topic_scores_0_10(raw)
# compute the final CHS score from the NORMALISED topic scores
chs = compute_CHS(topic_0_10)
#-------------------------
print("Per-topic (0-10):", {k: round(v,1) for k,v in topic_0_10.items()})
print("CHS:", round(chs, 1))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Per-topic (0-10): {'civic_engagement': 5.0, 'life_satisfaction': 7.4, 'community': 5.3, 'access_to_services': 5.1, 'income': 3.8, 'environment': 1.5, 'education': 4.2, 'jobs': 5.0, 'safety': 5.0, 'health': 5.0, 'housing': 5.0}
CHS: 4.7


In [4]:
# get the city names from api_1.json
import json

with open("api_1.json", "r", encoding="utf-8") as f:
    api_1 = json.load(f)

# extract the city names into a list of strings (preserve order, drop empty/duplicate)
city_names = [entry.get('city', '') for entry in api_1 if 'city' in entry]
seen = set()
city_names = [c for c in city_names if c and (c not in seen and not seen.add(c))]

print(f"Loaded {len(city_names)} cities")
# show a short sample to verify
city_names[:20]  # first 20 cities


Loaded 186 cities


['Reading',
 'Worth',
 'Liverpool',
 'Deal',
 'Boston',
 'Street',
 'York',
 'Glasgow',
 'Rugby',
 'Sale',
 'March',
 'Washington',
 'Tottenham',
 'Manchester',
 'Stone',
 'London',
 'Newcastle',
 'Middleton',
 'Hook',
 'Mold']

In [6]:
get_city_messages(10, cities=city_names)

[{'text': "Lebanon's parliamentary speaker, Nabih Berri (from Amal), said, in an interview with ad-Diyar, that the chances of Lebanon and Israel reaching a ceasefire agreement had surpassed 50% following president elect Donald Trump urging U.S. envoy Amos Hochstein to push through a deal."},
 {'text': 'Hey a daily 5 and a half mile walk is nothing to be sniffed at! Walking is such an underrated exercise! It’s fabulous. Plus it has the bonus of getting you outside into the fresh air too. It’s definitely worth doing 👏🏻'},
 {'text': 'There is no way he was ‘playing in the street’ - rolling about on the lawn with the Althorp kids maybe ?'},
 {'text': 'Stump for Trump\n-Sarah Palin\n\nChump for Trump\n-Track Palin\n\nBaby Bump for Trump\n-Bristol Palin\n\nSump Pump for Trump\n-Todd Palin'},
 {'text': 'You’re gonna need some derby girls. 🛼💪'},
 {'text': 'So, I wake to Biden asking for $24 Billion and this is good for two reasons.\n1. It is needed and the right thing.\n2. Will show how full o